<a href="https://colab.research.google.com/github/rzl-ds/gu511/blob/master/004_git.ipynb" target="_parent">
    <img src="https://colab.research.google.com/assets/colab-badge.svg"/>
</a>

# `git`

## why

Sharing code across several projects is messy. There are plenty of issues that can crop up:

you and a teammate removed, changed, and added many different things in the same file – how do you reconcile those changes?

*one person added a `scale` function:*

```python
def scale(col):
    return (col - col.min()) / (col.max() - col.min())

def process_data(df):
    df.loc[:, 'x'] = scale(df.x)
```

*the other added a second scaling step:*

```python
import numpy as np

def process_data(df):
    df.loc[:, 'x'] = (df.x - df.x.min()) / (df.x.max() - df.x.min())
    df.loc[:, 'y'] = np.log1p(df.y)
```

you made assumptions about how the program worked in one file and your teammate made different assumptions – now you are both developing code in different contexts. could you have avoided this?

*you* assumed input features would be standardized with mean 0 and a $\sigma$ of 1 and programmed your model contributions under that assumption

*your coworker* assumed they would be winsorized and then min-max scaled 0 to 1

you made changes to your project that the client didn't like and they want to "go back to how it was on Monday" – how do you go back?

uh.... why is almost everything getting scored above 50% risky all of a sudden?!? management will kill me.

```python
# this is an incredibly skewed distribution and dividing 
# by the mean was a bad idea, divide by the median instead
#df.x = df.x / df.x.mean()
df.x = df.x / df.x.median()
```

you want to try out something that may be a total failure without ruining the production codebase – how do you experiment?

```python
# what if it's a deep neural net but I make the activation
# functions ALSO be deep neural nets
# it's 4AM but I'm pretty sure this is genius
layer = tf.keras.Dense(10000, activation=netception)
```

**<div align="center">the ultimate reason: you are going to use it and people will want you to use it</div>**

## what

**"Version Control"** is the general concept of addressing the above issues (and more) in a systematic way -- a type of discipline or philosophy to help us develop code either on our own or collaboratively while also avoiding the above pitfalls.

**`git`** is one of several industry-standard tools used to implement the practice of version control in real development environments.

under the hood, `git` is a command-line tool which implements a graph-based approach to tracking different versions of files and the pathways (edits) from one version to the next.

<br><img width="1000px" src="https://miro.medium.com/max/552/1*PiduCtSA7kMwdPiMZo1nHw.jpeg"></img>

thinking of the edit history of files as directed graphs from one "snapshot" to the next allows us to consider both

1. the history of a given version of the file (all the changes that were made to "get to now")
1. other paths / parallel work done by multiple people (different "branches" of the "tree" of changes)

the various commands and subcommands associated with the main command `git` (e.g. `add`, `commit`, `merge`, `checkout`, `push`, `pull`; more details below) are all meant to be isolated, atomic ways of saying exactly what we're doing when we update code, specifically so that we can avoid the pitfalls mentioned above.

*short-term*: `git` is confusing! it's a new way of approaching collaboration, and it's easy to mess up.

*long-term*: the benefits are immense. the more people we have working on a project and the more often we are working together and independently, the more crucial `git` becomes to enforcing group communication

### one important distinction: local VC vs. remote VC

there is one distinction which I personally think is subtle but very valuable, so let's start with it: there are two different contexts for version control

**local** vs. **remote** version control

#### local

I, as a standalone developer, may want to use git on my own projects. problems 3 and 4 in the WHY section above (rolling back changes or tracking development changes) are both issues that occur even without collaboration. this is really where "version" is coming from – I want the ability to control the version of the code I have written, to create new versions and to move between old and new versions easily and with confidence

common local-specific commands

+ `add`
+ `commit`
+ `merge`
+ `branch`
+ `checkout`

#### remote

in addition to my personal code development, I might be collaborating as well. This is where problems 1 and 2 in the WHY section above (miscommunication due to simultaneous editing / not sharing full context) crop up. `git` gives us a way of communicating exactly what we changed and how, and formalizing the process of combining changes made by more than one person and communicating changes to a large group of people

common remote-specific commands

+ `fetch`
+ `pull`
+ `push`

advanced: technically, `pull` is a shorthand for the remote-only `fetch` and the local-only `merge`

## how

the best way to demonstrate how we use `git` is to actually use a couple of commands on a toy project. let's work on an important new Storyblocks project called

<div align="center"><h2><code>tacoworld</code></h2></div><br>

<br><div align="center"><img src="https://pbs.twimg.com/media/CpbLuCeWIAAMxSb.jpg:large" width="800"></div>

if you want to follow along, all you need to do is get into an environment where you have `git` installed. options:

1. log in to your `ec2` instance (the demo is tested for this option)
1. open the `terminal` app on your `mac` laptop
1. open `git` `bash` on your `windows` laptop

then, at the command line, enter:

```sh
mkdir -p ~/code/tacoworld
cd ~/code/tacoworld
```

I have crafted an amazing `python` script which predicts what my coworkers and I want for lunch. I have written my code to a file called `tacoworld.py` and placed it in its own `tacoworld` directory:

<br><div align="center"><img src="http://drive.google.com/uc?export=view&id=1ELbVinyk99LVPQ7U96pXTv0gqL2RvRVr"></div>

the code itself is proprietary and extremely advanced, so please do not share it beyond this class

```python
# file name: tacoworld.py

print("I want tacos!")
```

for those following along at home, create your own `tacoworld.py` with:

```sh
echo 'print("I want tacos!")' > tacoworld.py
```

### `git init`

I want to make sure that as I make improvements I can keep track of versions.

I know I will use `git` to do this, so the very first thing I want to do is make sure `git` knows I want to track what's going on in `tacoworld`.

I change directories into the `tacoworld` directory, and I use the `git` command line client to initialize my `git` repo:

```sh
cd ~/code/tacoworld
git init
```

```
Initialized empty Git repository in ~/code/tacoworld/.git/
```

*Note: you will only have to do this if you are starting from scratch – more often than not, you will be cloning – see below.*

### git status

`git` is happy to tell me what changes it believe have been made to my files since I started my repository, and it does so via the command

```sh
git status
```

```
On branch master

No commits yet

Untracked files:
  (use "git add <file>..." to include in what will be committed)
  
        tacoworld.py
        
nothing added to commit but untracked files present (use "git add" to track)
```

There's a bit to unpack here, so line by line:

```
On branch master
```

this means I am currently looking at a *branch* of the graph (will define later) called "master"

```
On branch master

No commits yet
```

I haven't told `git` to record (commit) any changes I've made to my code.

```
On branch master

No commits yet

Untracked files:
  (use "git add <file>..." to include in what will be committed)
  
        tacoworld.py
```

this heading indicates that `git` is about to list some files it has never yet seen, and if you want to record changes, you will need to "add" those files with the `git add` command

```
On branch master

No commits yet

Untracked files:
  (use "git add <file>..." to include in what will be committed)

	tacoworld.py
        
nothing added to commit but untracked files present (use "git add" to track)
```

finally, one last summary statement: there's nothing to commit, but also maybe something you *should* commit, nudge nudge

### `git add`

`git` said we should add so let's add!

to return to the "discipline" aspect of version control, this is a formal statement that I would like to put *something* into the list of changes I am making to my files. I'm not making this an "official" `git`-tracked change; I'm just getting ready to do it

again, using the command line:

```sh
git add tacoworld.py
```

`git` itself won't say anything about this change, but it will tell you something new and different if you ask for the status again

```sh
git status
```

```
On branch master

Initial commit

Changes to be committed:
  (use "git rm --cached <file>..." to unstage)

         new file:   tacoworld.py

```

`tacoworld.py` was previously "untracked" and is now listed as a "new file" with "changes to be committed" -- that's about right.

*advanced*: for future `git add` calls, I may decide to use a flag: `-p` or `--patch` to "pick" "patches" of changes. this allows me to see what exactly changed and decide whether or not I want to add that change to my current snapshot.

### `git commit`

you may think we're done now, but actually we're only halfway. `git` separates out the act of declaring what changes you make (`git add`) from the act of officially recording them as a "change" to the code base (`git commit`).

a useful analogy: think of the process of painting a still-life painting. before you can paint the picture, you have to assemble the pieces you want to paint.


<br><div align="center"><img src="https://upload.wikimedia.org/wikipedia/commons/8/8a/Willem_Claesz._Heda_-_Still-Life_with_Pie%2C_Silver_Ewer_and_Crab_-_WGA11248.jpg" width="500"></div>

a useful analogy: think of the process of painting a still-life painting. before you can paint the picture, you have to assemble the pieces you want to paint.


<br><div align="center"><img src="https://cdn.shopify.com/s/files/1/0817/6361/products/01062-Still_20Life_20With_20Pink_20Vase_large.jpeg?v=1465396397" width="500"></div>
<em><div align="center">quality of portrait may vary</div></em>

we've added all the pieces we wanted to keep track of -- let's make it facebook official. let's `commit` them.

in doing this, we are creating a *snapshot* of the code at a certain point in time (literally: we are saving a copy of the real files every time we `commit`).

from now on, no matter what we do we will always be able to get back to this point in the history of the code.

we'll add a "commit message" -- a short text summary of what we changed -- with the `-m` (`--message`) flag

```sh
git commit -m 'tacoworld: initial commit'
```

```
[master (root-commit) 14caba7] tacoworld: initial commit
 1 file changed, 3 insertions(+)
 create mode 100644 tacoworld.py
```

*note*: if you received the following:

```
*** Please tell me who you are.

Run

  git config --global user.email "you@example.com"
  git config --global user.name "Your Name"

to set your account's default identity.
```

go ahead and run those two `git config` commands (updating for your real email address and name, of course) and try committing again

we can verify that we're "current" with a call to `git status`:

```sh
git status
```

```
On branch master
nothing to commit, working tree clean
```

### another cycle: edit, `git status`, `git add`, `git status`, `git commit`

let's put this all together for a full development cycle. we know we're current (just checked). let's edit our file!

suppose I make a change to `tacoworld.py` to be a bit bolder:

```python
# file name: tacoworld.py

print("I, Zach Lamberty, want tacos!!")
```

*for those following along, run:*

```
echo 'print("I, Zach Lamberty, want tacos!")' > tacoworld.py
```

*feel free to use your own name*

`type type type type... save`. now that I've saved those changes, and now I want to know what `git` sees.

```sh
git status
```

```
On branch master
Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   tacoworld.py

no changes added to commit (use "git add" and/or "git commit -a")
```

in particular, note that `new file` was replaced with `modified` (we're tracking changes, and saw one!), and that we have a new message about using `git checkout` to discard those changes and revert to our last saved state -- not bad.

let's add that change -- this time using the `-p` flag to "pick" "patches" of changes (you can skip this by *not* providing the flag)

```sh
git add -p tacoworld.py
```

```
diff --git a/tacoworld.py b/tacoworld.py
index 77e8759..214d68d 100644
--- a/tacoworld.py
+++ b/tacoworld.py
@@ -1 +1 @@
-print("I want tacos!")
+print("I, Zach Lamberty, want tacos!")
Stage this hunk [y,n,q,a,d,/,e,?]? y
```

again, checking to see what `git` sees

```sh
git status
```

```
On branch master
Changes to be committed:
  (use "git reset HEAD <file>..." to unstage)
  
        modified:   tacoworld.py
        
```

and `commit`-ing those changes:

```sh
git commit -m 'tacoworld: my name is Zach and I do not fear my hunger'
```

```
[master b115876] tacoworld: my name is Zach and I do not fear my hunger
 1 file changed, 1 insertion(+), 1 deletion(-)
```

let's go back to that middle stage where we called `git add` with the extra `-p` flag:

```sh
git add -p tacoworld.py
```

```
diff --git a/tacoworld.py b/tacoworld.py
index 77e8759..214d68d 100644
--- a/tacoworld.py
+++ b/tacoworld.py
@@ -1 +1 @@
-print("I want tacos!")
+print("I, Zach Lamberty, want tacos!")
Stage this hunk [y,n,q,a,d,/,e,?]? y
```

this jumbly bunch of text and symbols is a very compact way of isolated what *really* changed from one version of `tacoworld.py` to the next.

`git` is extremely efficient at comparing the last recorded version of the code to the current state and calculating the `diff`s: standardized text representations of changes, additions, and deletions to lines in the now-edited files.

```
diff --git a/tacoworld.py b/tacoworld.py
index 77e8759..214d68d 100644
--- a/tacoworld.py
+++ b/tacoworld.py
@@ -1 +1 @@
-print("I want tacos!")
+print("I, Zach Lamberty, want tacos!")
Stage this hunk [y,n,q,a,d,/,e,?]? y
```

`git` noticed that the original version of line 1 in my old file `print("I want tacos!")` changed into a different string in the new file `print("I, Zach Lamberty, want tacos!")`.

`git` asked if I wanted to include this change (`Stage this hunk?`) and I said that `y`es I did want to add that change.

### `git log`

we've now made two `commi`s to our `git` repository. I told you that one reason to use version control is to have a running list of the changes we've made to our files; how do we see that list?

`git` records a history of changes to files there is a `git log` -- a history of all the batches of changes we've ever recorded. changes to files are listed in a newest-to-oldest order.

```sh
git log
```

```
commit b115876f2a8c0c7014c2b0bacfe273e2e487095a (HEAD -> master)
Author: r.zach.lamberty <r.zach.lamberty@gmail.com>
Date:   Mon Sep 10 22:08:13 2018 -0400

    tacoworld: my name is Zach and I do not fear my hunger

commit 29ae0207399bf909ab33cb47c26875399aa35c7c
Author: r.zach.lamberty <r.zach.lamberty@gmail.com>
Date:   Mon Sep 10 22:07:25 2018 -0400

    tacoworld: initial commit
```

this log message is a collection of different `commit`s represented as a block of text. each block of text contains

1. the `commit` `sha`: this is a unique 40-digit character string that is the [`SHA-1` hash](https://en.wikipedia.org/wiki/SHA-1) of the current state of the file (there is a lot of hand waving here)
    + basically, think of this as a unique lookup key that `git` has associated with that state in time
1. metadata: the author and date information for the commit
1. message: the message I recorded when I added that commit

generally, that command is overkill and we really only one one line of information. you'll never guess what the flag is that gives us only one line of information...

also, I'll add the `--decorate` flag because it adds some information that is frequently useful

```sh
git log --oneline --decorate
```

```
b115876 (HEAD -> master) tacoworld: my name is Zach and I do not fear my hunger
29ae020 tacoworld: initial commit
```

there is also a super crazy long command that I have recorded permanently as a fancy alias (see the "advanced topics" below):

```sh
git log --graph --abbrev-commit --decorate --date=relative --format=format:'%C(bold blue)%h%C(reset) - %C(bold green)(%ar)%C(reset) %C(bold white)%s%C(reset) - %C(red)%an%C(reset)%C(bold yellow)%d%C(reset)' --all
```

```
* b115876 - (5 minutes ago) tacoworld: my name is Zach and I do not fear my hunger - r.zach.lamberty (HEAD -> master)
* 29ae020 - (6 minutes ago) tacoworld: initial commit - r.zach.lamberty
```

To be honest, I pretty much never use the basic `git log`, I only use the alias for that long command.

### `HEAD -> master`

you may have noticed that in all of the `git log` commands above there was the text

```
(HEAD -> master)
```

one quick note on this: `git` uses a special human-readable set of tags to describe some of those `SHA` values. in particular, `HEAD` and `master` are two of those tags.

we will revisit these in `git branch` below when it will hopefully make more sense why these tags are useful

### quick summary of "local" commands

so far we've done all our work on our local computer, and we could keep going like this forever:

+ `git init`

then we loop through the following:

+ edit code
+ `git status`
+ `git add`
+ `git commit`

and also, `git log` if we want history

**<div align="center">what are your questions so far??</div>**

### sharing with the world: `github`

`git` is very useful for monitoring your own code's version history, but it becomes invaluable when you are working with others. rather than go through the complicated back-and-forth of explaining what changed and how, let `git` handle it for you!

in order to share code, we need to put it in some shared place (the jargon: we need a mutually accessible file share for our `bare` repositories).

fortunately, there are well-excepted tools for doing exactly this. the primary ones:

+ [`github`](https://github.com/)
+ [`gitlab`](https://about.gitlab.com/)
+ [`bitbucket`](https://bitbucket.org/)

let's use `github`! it's free and hosted for us and why not?

you all created `github` accounts for a previous homework, so go to https://github.com and log in.

once logged in, click on the plus sign in the top right menu bar and select "new repository"

<br><div align="center"><img src="http://drive.google.com/uc?export=view&id=13w1X-pialhgAkGBH7sak2FS4FnLlLOCh" width="600"></div>

<br><div align="center"><img src="http://drive.google.com/uc?export=view&id=1Zj0IUYuFgvt_339uM8UAPkUnxDZeFdt5" width="850"></div>

**copy, but don't paste!!**

<br><div align="center"><img src="http://drive.google.com/uc?export=view&id=1_V2LmtLOUdSHBCAlKE7r1GRvj6GlJ_h4" width="850"></div>

at this point we have created a *place* where we could share our code with the world. the commands we just copied are the commands that will actually share it.

rather than just copy/paste-ing that, let's talk about what those commands are doing

### `git remote`

we use the `git remote` command to formally declare a "remote" place we would like to publish our code (to have an independent backup? to share? whatever the reason!). along the way, we give it a short nickname -- the default is "origin".

remember: at the top of that `github` page there was a toggle button to switch between `https` and `ssh`. those two options correspond to two different ways of communicating with `github`. depending on which we would have selected, we'd have two different `git remote` commands:

```sh
git remote add origin https://github.com/RZachLamberty/tacoworld.git
git remote add origin git@github.com:RZachLamberty/tacoworld.git
```

the only difference is the connection url, and depending on which we choose we will have a different authentication scheme when we try to talk to `github`:

| url | protocol | authentication method |
|-|-|-|
| `https://github.com/RZachLamberty/tacoworld.git` | `https` | `github` user name and password login (every time!) |
| `git @github.com:RZachLamberty/tacoworld.git` | `ssh` | `ssh` public / private key exchange |

head on back over to the command line and enter the first of those two commands (for `ssh` access):

```sh
git remote add origin git@github.com:RZachLamberty/tacoworld.git
```

now, whenever you write "origin", `git` knows to look to your `github` repo. cool.

that's only part of the battle, though -- we've told `git` what url to look at when we want to talk using `ssh`, but what are the other two things we always have to provide (in addition to `url`) to talk via `ssh`?

1. username (assumed by `github` to be any one of the users who own the project)
1. **authentication** in the form of a public / private key exchange

### adding an `ssh` key to your `github` account

in order to `push` code from your local machine (your laptop) to `github`, you need to give `github` your public (***NOT PRIVATE***) key.

first, let's determine whether or not you have a key pair. on our `ec2` instance, type:

```sh
ls ~/.ssh/id_rsa*
```

if you got a pair of file names, sit tight.

if instead you received a "no such file or directory" error, let's *create* a key pair on our `ec2` instance. start with

```sh
ssh-keygen -t rsa -b 4096 -C <YOUR GU ID>@georgetown.edu
```

and keep pressing `enter` until you're done

now everyone should be able to enter the following command and have the public key printed to the screen:

```sh
cat ~/.ssh/id_rsa.pub
```

go ahead and copy that. then do steps 2 - 8 of [this walkthrough](https://help.github.com/articles/adding-a-new-ssh-key-to-your-github-account/)

so we did that...

can anyone take a step back here and remind us *why* we did that? what are we doing right now?

### `git push`

way way back, `github` asked us to run the following two commands:

```sh
git remote add origin git@github.com:RZachLamberty/tacoworld.git
git push -u origin master
```

we already ran `git remote` and explained what that command is doing, and *also* added `ssh` keys to our `github` profile so that we are allowed to talk to that particular url (`git@ github.com:<user name>/tacoworld.git`) as a `remote`. now, it's time to `push`

```sh
git push -u origin master
```

1. the `-u` flag is setting an "upstream" -- don't worry about this just yet
1. we specifically tell `git` to use the `remote` nicknamed `origin` we created above. this would have happened by default for reasons we will get into later
1. the "branch" name `git` should use is `master` -- more on this later

if everything went according to plan:

```
Counting objects: 6, done.
Delta compression using up to 4 threads.
Compressing objects: 100% (2/2), done.
Writing objects: 100% (6/6), 529 bytes | 529.00 KiB/s, done.
Total 6 (delta 0), reused 0 (delta 0)
remote: 
remote: Create a pull request for 'master' on GitHub by visiting:
remote:      https://github.com/RZachLamberty/tacoworld/pull/new/master
remote: 
To github.com:RZachLamberty/tacoworld.git
 * [new branch]      master -> master
Branch 'master' set up to track remote branch 'master' from 'origin'.
```

go back to your browser with those commands we copied and refresh

<br><div align="center"><img src="http://drive.google.com/uc?export=view&id=16zJp6aN4NsdrX9FcY7HHa3l3eeZahXLD" width="850"></div>

in particular, go check out that "commits" link:

<br><div align="center"><img src="http://drive.google.com/uc?export=view&id=1ABZ6xCGTuhjSH_Qayblt3Gku17OcVjZb" width="850"></div>

### summary on *establishing* a `remote`

the `ssh` key setup process we just went through is something that has to happen *once* for *each laptop* you want to use. your public and private key uniquely identify you on that machine.

the `github` interface "plus button" repo creation step as well as the copied commands:

```sh
git remote add origin git@github.com:RZachLamberty/tacoworld.git
git push -u origin master
```

happen *once* for *each* new repo

### collaboration using official `github` collaborators

using `github`, there are a few ways to collaborate. the first we'll talk about -- and the one that is closer to how non-`github` implementations of `git` remote repositories work -- is by adding other users as "collaborators" to this specific repo (the other is via forks and pull requests; we'll talk about that below).

but for now, collaborators:

I posted the preliminary findings of my predictive model and the DC data science community is buzzing.

in particular, your TA Dan is interested and wants to collaborate. I add him as a collaborator on my repositroy:

<br><div align="center"><img src="http://drive.google.com/uc?export=view&id=1TwrmqqP64OvN2lrLO_OiQMLXi31gaLL1" width="850"></div>

### `git clone`

the first thing he does as a collaborator is get a *local* copy on his laptops of the shared *remote* repository on `github`. he does this using `git clone` and the url from before (he gets this via `github`).

<br><div align="center"><img src="http://drive.google.com/uc?export=view&id=1EMhdQ1fZHyhI7vrbRMgrn9kg5lAGjeSR" width="850"></div>

then, at the command line he types:

```sh
git clone https://github.com/RZachLamberty/tacoworld.git
```

```
Cloning into 'tacoworld'...
remote: Counting objects: 6, done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 6 (delta 0), reused 6 (delta 0), pack-reused 0
Unpacking objects: 100% (6/6), done.
Checking connectivity... done.
```

this creates a local directory named `tacoworld` and downloads the `git` history, then builds the current version of the files inside that directory

### dan is hungry too: edit, `git status`, `git add`, `git commit`, `git push`

dan wants to make his preferences known, so he repeats the `git` version update cycle. he makes his changes to the file:

```python
# file name: tacoworld.py

print("I, Zach Lamberty, want tacos!!")
print("I, dan fein, want tacos!!")
```

```sh
git status
```

```
On branch master
Your branch is up-to-date with 'origin/master'.
Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   tacoworld.py

no changes added to commit (use "git add" and/or "git commit -a")
```

```sh
git add -p
```

```
diff --git a/tacoworld.py b/tacoworld.py
index 214d68d..1a0db65 100644
--- a/tacoworld.py
+++ b/tacoworld.py
@@ -1 +1,2 @@
 print("I, Zach Lamberty, want tacos!")
+print("I, dan fein, want tacos!")
Stage this hunk [y,n,q,a,d,/,e,?]? y
```

```sh
git commit -m 'tacoworld: I, dan, also desire tacos'
```

```
[master 9c55fa3] tacoworld: I, dan, also desire tacos
 1 file changed, 1 insertion(+)
```

### another `git push`

dan is proud of his work and wants to share it with the team. the way he does that is by pushing those changes to the shared repository. `git push` takes the log of individual file changes (the "commit log", the `diff`s we saw when we ran `git add -p`) and updates the remote (shared) log with those changes.

```sh
git push origin master
```

```
Counting objects: 3, done.
Compressing objects: 100% (2/2), done.
Writing objects: 100% (3/3), 330 bytes | 0 bytes/s, done.
Total 3 (delta 0), reused 0 (delta 0)
To https://github.com/RZachLamberty/tacoworld.git
   b115876..9c55fa3  master -> master
```

and check out `github` after that

<br><div align="center"><img src="http://drive.google.com/uc?export=view&id=1XWjZuVvVf4mexuYgz6XBG7wxdnNObc1s" width="850"></div>

so far we've discussed two ways of *initializing* a repository on your local laptop:

1. `git init` to "start from local", i.e. create your own from scratch, and
2. `git clone` to "start from remote", i.e. download a repository someone else already started and shared on a remote like `github`

we also have discussed a way to *add* changes (`commit`s) to those repositories once you have them: `git push`

we *haven't* talked about how to *GET* changes other people have made to shared repositories.

### `git pull`

the opposite of `push` is `pull`, and that's what we will use to download updates our collaborators have added to our shared `github` repo.

in this case I happen to *know* dan made the last updates, so I need to update my local version to incorporate his changes. I won't always know when changes have been made, but that doesn't have to affect my workflow -- that's the point of using `git`

```sh
# note: I don't have to write out all of
#   git pull origin master.
# I could, but both of the "origin master" values
# are assumed by default
git pull
```

```
From github.com:RZachLamberty/tacoworld
   b115876..9c55fa3  master     -> origin/master
Updating b115876..9c55fa3
Fast-forward
 tacoworld.py | 1 +
 1 file changed, 1 insertion(+)
```

under the hood, `git pull` is doing two things:

1. it downloads the current state of the `remote` (this is `git fetch`, which downloads the current remote `master` state to `origin/master`)
1. it incorporates the changes that have been made to `master` in that `remote` to your current `master` (`git merge origin/master`)

we can verify that the code I have includes dan' changes:

```sh
cat tacoworld.py
```

```python
print("I, Zach Lamberty, want tacos!")
print("I, dan fein, want tacos!")
```

### making changes

having just run `git pull`, I know that my local state is current.

I want to make some changes, so I follow the same workflow:

1. edit
1. `git add`
1. `git commit`
1. `git push`

I edit my `tacoworld.py` file to make sure that everyone understands my full commitment to and trust in my predictive model. the new version is:

```python
print("I, Zach Lamberty, want tacos more than *anyone*, especially more than dan!")
print("I, dan fein, want tacos!")
```

```sh
git add -p
```

```
diff --git a/tacoworld.py b/tacoworld.py
index 1a0db65..d14f5b5 100644
--- a/tacoworld.py
+++ b/tacoworld.py
@@ -1,2 +1,2 @@
-print("I, Zach Lamberty, want tacos!")
+print("I, Zach Lamberty, want tacos more than *anyone*, especially more than dan!")
 print("I, dan fein, want tacos!")
Stage this hunk [y,n,q,a,d,/,e,?]? y
```

```sh
git commit -m 'tacoworld: v cool good for you dan but I wanted tacos first bruh so back off'
```

```sh
git push
```

```
Counting objects: 3, done.
Delta compression using up to 4 threads.
Compressing objects: 100% (2/2), done.
Writing objects: 100% (3/3), 394 bytes | 394.00 KiB/s, done.
Total 3 (delta 0), reused 0 (delta 0)
To github.com:RZachLamberty/tacoworld.git
   9c55fa3..7241432  master -> master
```

and lo!

<br><div align="center"><img src="http://drive.google.com/uc?export=view&id=1lfnkxOMYVzWDNkh5klTOL3pDUdPT-DvO" width="850"></div>

**<div align="center">what are your questions so far??</div>**

### keep on' summarizing

so we have talked about how to track changes locally:

1. `git init` to create a repo
1. `git add` and `git add -p` to *stage* changes
1. `git commit` and `git commit -m` to *commit* changes

and we have also talked about how to communicate those changes with the outside world

1. `git remote add` and `git remote -v` to add or list shared "remote" repositories
1. `git clone` to initially download the current state of a shared repository
1. `git pull` to pull down (download) *changes* other people have recorded in the shared repository
1. `git push` to push up (upload) *changes* you have recorded in the shared repository 

### `git checkout`

we call it version *control*, not version *tracking* -- this implies that we should be able to not only record what's changed, but actually "move around" to different points in the code history.

well, we can, using the command `git checkout`

right now, `tacoworld.py` looks like:

```sh
cat tacoworld.py
```

```py
print("I, Zach Lamberty, want tacos more than *anyone*, especially more than dan!")
print("I, dan fein, want tacos!")
```

suppose we want to "go back" to the code before dan tried to show off about how much he likes tacos when it was *MY* idea in the first place.

to start, let's look at the history of changes so far:

```sh
git log --oneline --decorate
```

```
7241432 (HEAD -> master, origin/master) tacoworld: v cool good for you dan but I wanted tacos first bruh so back off
9c55fa3 tacoworld: I, dan, also desire tacos
b115876 tacoworld: my name is Zach and I do not fear my hunger
29ae020 tacoworld: initial commit
```

currently there are two tags `master` and `origin/master` which are associated with `sha` `7241432`. these are just human-readable aliases for that 40-character `sha` string.

there is also a special tag called `HEAD` which is telling me that my current "working state" is at `7241432` as well.

`HEAD` can move around in this log!

I want to go back before dan ruined things. the last state I liked was `b115876` we can `checkout` that point in time using `git checkout`:

```sh
# use your own sha values if you're following
git checkout b115876
```

```
Note: checking out 'b115876'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>

HEAD is now at b115876 tacoworld: my name is Zach and I do not fear my hunger
```

look at the log again, this time throwing in the `--all` flag:

```sh
git log --oneline --all
```

```
7241432 (origin/master, otr, master) tacoworld: v cool good for you dan but I wanted tacos first bruh so back off
9c55fa3 tacoworld: I, dan, also desire tacos
b115876 (HEAD) tacoworld: my name is Zach and I do not fear my hunger
29ae020 tacoworld: initial commit
```

finally, look at the file:

```sh
cat tacoworld.py
```

```py
print("I, Zach Lamberty, want tacos!")
```

just like old times!

so, I've taken a few minutes to think things over and I've decided that *maybe* I overreacted. let's go back to the current state I shared with everyone already:

```sh
# when you have a tag, you can use that instead of the sha!
git checkout master
```

```
Previous HEAD position was b115876 tacoworld: my name is Zach and I do not fear my hunger
Switched to branch 'master'
Your branch is up to date with 'origin/master'.
```

```sh
git log --oneline --decorate
```

```
7241432 (HEAD -> master, origin/master) tacoworld: v cool good for you dan but I wanted tacos first bruh so back off
9c55fa3 tacoworld: I, dan, also desire tacos
b115876 tacoworld: my name is Zach and I do not fear my hunger
29ae020 tacoworld: initial commit
```

## `git branch`

sometimes you want to make changes but you want to keep them apart from the main code base

1. experiments that you want to *track* but might not think are keepers (a new modelling technique you haven't tried before)
1. features that will require a lot of time or effort and the code will work fine without them (e.g. adding an ensemble method, or a faster ETL process)
1. your team has separated out the development work into teams and your team is focused on one set of code changes

in situations like this, you may want to create a `branch`. this is a separately recorded set of changes -- a new "branch" in the "tree" of changes.

up until now, we've done everything in linear order, so this "tree" has been very boring -- just straight up, one list of sequential commits.

I just peaked out the windows and -- folks --

<div align="center"><h2>a challenger approaches</h2></div><br>

<br><div align="center"><img src="http://nebula.wsimg.com/55172a3ebc05eb20813565186dc56825?AccessKeyId=036259FD3A79DC4CD9B5&disposition=0&alloworigin=1" width="700"></div>

what happens if, some day, I'm tired of tacos and this wicked wagon shows up? the model must be updated to incorporate this new information. I think a `branch` is in order

we start by creating a new `branch` -- this is a formal declaration to `git` that we are going to create a new path to track. `git` will save this path separately

```sh
# otr = over the rice
git branch otr
```

first, let's note that this just *creates* the `branch` and doesn't do anything else:

```sh
# with no argument, it lists:
git branch
```

```
* master
  otr
```

furthermore, the output of `git log` tells us that `otr` is "at the same point" in the edit history as `master` and `origin/master`:

```sh
git log --oneline --decorate
```

```
7241432 (HEAD -> master, origin/master, otr) tacoworld: v cool good for you dan but I wanted tacos first bruh so back off
9c55fa3 tacoworld: I, dan, also desire tacos
b115876 tacoworld: my name is Zach and I do not fear my hunger
29ae020 tacoworld: initial commit
```

our `HEAD` tag (which represents our current working state in the edit history) is pointing at the `master` tag.

well guess what?

**`master` was a `branch` all along**

`master` is the default `branch`

switching to a different branch is done using `git checkout` (the command we used before to move to different points in the `master` branch / code edit history).

```sh
git checkout otr
```

```
Switched to branch 'otr'
```

note that `HEAD` is pointing at our `otr` branch tag now too:

```sh
git log --oneline --decorate
```

```
7241432 (HEAD -> otr, origin/master, master) tacoworld: v cool good for you dan but I wanted tacos first bruh so back off
9c55fa3 tacoworld: I, dan, also desire tacos
b115876 tacoworld: my name is Zach and I do not fear my hunger
29ae020 tacoworld: initial commit
```

note the before / after difference:

```
# before
7241432 (HEAD -> master, origin/master, otr) tacoworld: v cool good for you dan but I wanted tacos first bruh so back off

# after
7241432 (HEAD -> otr, origin/master, master) tacoworld: v cool good for you dan but I wanted tacos first bruh so back off
```

we have now:

+ created an isolated new place to track changes
    + named it something related to our "feature"
+ checked it out

from here, we can repeat the same old development cycle (edit, `add`, `commit`, `push`)

I edit the file to read:

```python
print("Today, actually, I (Zach Lamberty), want OTR!")
print("I, dan fein, want tacos!")
```

I add the changes:

```sh
git add -p
```

```
diff --git a/tacoworld.py b/tacoworld.py
index d14f5b5..ca3cfd7 100644
--- a/tacoworld.py
+++ b/tacoworld.py
@@ -1,2 +1,2 @@
-print("I, Zach Lamberty, want tacos more than *anyone*, especially more than dan!")
+print("Today, actually, I (Zach Lamberty), want OTR!")
 print("I, dan fein, want tacos!")
Stage this hunk [y,n,q,a,d,e,?]? y
```

I commit the changes:

```sh
git commit -m 'otr: what happens when over the rice is here?'
```

```
[otr c5043f8] otr: what happens when over the rice is here?
 1 file changed, 1 insertion(+), 1 deletion(-)
```

now, I `push`...

**EXCEPT**

remember, the `git push` command has two arguments it expects:

```sh
git push [remote] [branch (technically, a refspec)]
```

previously we have either been explicit in writing

```sh
git push origin master
```

or we have left off the `origin master` arguments entirely because the default `remote` and `branch` (`refspec`) values are set when the branch is created.

when the `master` branch was created, the `remote` defaults to `origin`, and the default `branch` value is `master` (I *told* you `master` was a `branch`)

*this time*

+ we are still publishing our results to the same place (`github`), so our `remote` is still the alias `origin` which points to `github`
+ the `branch` we are pushing to is new and called `otr`

```sh
# note the "origin otr" instead of "origin master"
git push origin otr
```

```
Total 0 (delta 0), reused 0 (delta 0)
remote: 
remote: Create a pull request for 'otr' on GitHub by visiting:
remote:      https://github.com/RZachLamberty/tacoworld/pull/new/otr
remote: 
To github.com:RZachLamberty/tacoworld.git
 * [new branch]      otr -> otr
```

meanwhile, back [on `github`](https://github.com/RZachLamberty/tacoworld):

<br><div align="center"><img src="http://drive.google.com/uc?export=view&id=1Lcv_LF2VFHk5eZlBGtZzp_iWNtIkX7Q-" width="850"></div>

locally, what has happened is perhaps easiest to see in the log:

```sh
git log --oneline --decorate --all
```

```
c5043f8 (HEAD -> otr, origin/otr) otr: what happens when over the rice is here?
7241432 (origin/master, master) tacoworld: v cool good for you dan but I wanted tacos first bruh so back off
9c55fa3 tacoworld: I, dan, also desire tacos
b115876 tacoworld: my name is Zach and I do not fear my hunger
29ae020 tacoworld: initial commit
```

+ our `HEAD` (current working state) is synce up with the branch we have checked out, `otr`
+ `otr` and `origin/otr` are synced up because we have pushed our local changes to `origin`
+ the `otr` branch is one `commit` ahead of the `master` branch

### diverging code paths

while I've been off preparing for OTR contingencies, dan has been trying to find a way to defuse the situation with his difficult coworker. he's decided to make a code change in the hope of resolving some of the tension.

at this point, the workflow is old hat:

+ `pull` to get up to date
+ check the `log` to see what changed
+ edit
+ `add` to stage changes
+ `commit` to commit them to record
+ `push` to share them with the world

**what happens when dan `git pull`s??**

it depends! what is the exact command he uses?

```sh
git pull
git pull origin master
git pull origin otr
git pull --all
```

dan isn't interested in OTR at this point so he just does the basic thing

```sh
# he has master checked out locally, so this
# command implies `origin master` as its arguments
git pull
```

```
Updating 9c55fa3..7241432
Fast-forward
 tacoworld.py | 2 +-
 1 file changed, 1 insertion(+), 1 deletion(-)
```

```sh
git log --oneline --decorate
```

```
7241432 (HEAD -> master, origin/master, origin/HEAD) tacoworld: v cool good for you dan but I wanted tacos first bruh so back off
9c55fa3 tacoworld: I, dan, also desire tacos
b115876 tacoworld: my name is Zach and I do not fear my hunger
```

yikes...

he updates the `tacoworld.py` file to read

```python
print("I, Zach Lamberty, want tacos more than *anyone*, especially more than dan!")
print("I, dan fein, want tacos, so it's good there are enough to share!")
```

```sh
git add -p
```

```
diff --git a/tacoworld.py b/tacoworld.py
index d14f5b5..168e4a3 100644
--- a/tacoworld.py
+++ b/tacoworld.py
@@ -1,2 +1,2 @@
 print("I, Zach Lamberty, want tacos more than *anyone*, especially more than dan!")
-print("I, dan fein, want tacos!")
+print("I, dan fein, want tacos, so it's good there are enough to share!")
Stage this hunk [y,n,q,a,d,/,e,?]? y
```

```sh
git commit -m 'tacoworld: plenty o tacos'
```

```
[master 043e801] tacoworld: plenty o tacos
 1 file changed, 1 insertion(+), 1 deletion(-)
```

```sh
# note: origin master is implied here again!
git push
```

```
Counting objects: 3, done.
Compressing objects: 100% (2/2), done.
Writing objects: 100% (3/3), 325 bytes | 0 bytes/s, done.
Total 3 (delta 1), reused 0 (delta 0)
remote: Resolving deltas: 100% (1/1), completed with 1 local object.
To https://github.com/RZachLamberty/tacoworld.git
   7241432..043e801  master -> master
```

and just like that

we have

a fork

<div align="center"><img src="https://image.freepik.com/free-icon/code-fork-symbol_318-55528.jpg" width=400></div>

### `git fetch`, `git merge`, and "local remote" `branch`es

okay, let's seriously level up our `git` knowledge.

I mentioned up above that a `git pull` was really a combination of two steps:

+ a "remote" command `git fetch`
+ a "local" command `git merge`

let's dig into that.

I pull up the `log` again:

```sh
git log --oneline --decorate --all
```

```
c5043f8 (HEAD -> otr, origin/otr) otr: what happens when over the rice is here?
7241432 (origin/master, master) tacoworld: v cool good for you dan but I wanted tacos first bruh so back off
9c55fa3 tacoworld: I, dan, also desire tacos
b115876 tacoworld: my name is Zach and I do not fear my hunger
29ae020 tacoworld: initial commit
```

whenever we've actually explicitly typed out commands involving the phrases `origin`, `master`, and `otr`, there has been a space between them:

```sh
git pull origin master
git push origin otr
```

but those tags have a **slash**:

```
origin/master
origin/otr
```

what if I told you that those tags were actually *local* `branch`es tracking *remote* `branch`es, and you were really syncing up with `origin/master` instead of `origin master` this whole time

<br><div align="center"><img src="http://gif-finder.com/wp-content/uploads/2015/12/Tim-Eric-Mind-Blown.gif" width="480"></div>

let's break this down.

although `git` *could* constantly be checking on `origin` (e.g. `github`) for changes, it *isn't*.

it only checks for changes when you tell it to, and the way it does that is with a command called `git fetch`.

`git fetch` does the following:

+ sends a message out over the internet to the `remote` (`github`) asking if anything changed
+ `github` responds with a "no" or with new `commit`s
+ takes new `commit`s and updates a *LOCAL* branch that is *VERY SPECIAL*, and designed to be completely locked to the state of the `remote` (`github`)
    + unlike local branches you create, you can't make edits to this special branch
    + the name is, by default, `[origin name]/[branch or refspec name]`, so we have two such branches now: `origin/master` and `origin/otr`

we have two local branches we are editing (`master` and `otr`) and two special local branches which are tracking `remote`s (conveniently called "remote branches" because that's the most confusing thing to call them).

we can update both of the tracking branches with

```sh
git fetch --all
```

```
Fetching origin
remote: Counting objects: 3, done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 3 (delta 1), reused 3 (delta 1), pack-reused 0
Unpacking objects: 100% (3/3), done.
From github.com:RZachLamberty/tacoworld
   7241432..043e801  master     -> origin/master
```

I recorded the output of `git log --online --decorate --all` before and after that command:

```
c5043f8 (HEAD -> otr, origin/otr) otr: what happens when over the rice is here?
7241432 (origin/master, master) tacoworld: v cool good for you dan but I wanted tacos first bruh so back off
9c55fa3 tacoworld: I, dan, also desire tacos
b115876 tacoworld: my name is Zach and I do not fear my hunger
29ae020 tacoworld: initial commit
```

and after

```
043e801 (origin/master) tacoworld: plenty o tacos
c5043f8 (HEAD -> otr, origin/otr) otr: what happens when over the rice is here?
7241432 (master) tacoworld: v cool good for you dan but I wanted tacos first bruh so back off
9c55fa3 tacoworld: I, dan, also desire tacos
b115876 tacoworld: my name is Zach and I do not fear my hunger
29ae020 tacoworld: initial commit
```

the `fetch` command was informed by `github` of the only thing that dan changed on remote -- the `master` branch -- and as a result it updated the *local remote branch* `origin/master`

**<div align="center">you with me so far? what are your questions?</div>**

now like I said, `git pull` is a combo of `git fetch` and `git merge`. what is `git merge`?

#### `git merge`

every `branch` has a most recent `commit`

```
043e801 (origin/master) tacoworld: plenty o tacos
c5043f8 (HEAD -> otr, origin/otr) otr: what happens when over the rice is here?
7241432 (master) tacoworld: v cool good for you dan but I wanted tacos first bruh so back off
9c55fa3 tacoworld: I, dan, also desire tacos
b115876 tacoworld: my name is Zach and I do not fear my hunger
29ae020 tacoworld: initial commit
```

+ `origin/master` points to `043e801`
+ `otr` and `origin/otr` point to `c5043f8`
+ `master` points to `7241432`

`git merge` is a command which will take your current active state (the most recent commit of your current branch) and try and apply the `commit` history of another point in the `git` commit graph (that is, all the changes made to get to that file state) on top of your current active state.

the `ascii` diagrams in the `git merge --help` manual are useful here:

if we have `master` checked out and run `git merge topic`, that will make the following schematic updates:

```
      A---B---C topic
     /
D---E---F---G master
```

becomes

```
      A---B---C topic
     /         \
D---E---F---G---H master
```

note that this is the most general case and results in two different paths "merging" (hence the name).

a very common case is one in which one branch (e.g. `origin/master`) sits *directly* ahead of another (e.g. `master`). this is called a "fast forward" merge.

let's check out `master` and look at the log, this time with `--graph` to make beautiful `ascii` art

```sh
git checkout master
git log --oneline --decorate --graph --all
```

```
# helpful hint from `git` that we're behind, first:
Your branch is behind 'origin/master' by 1 commit, and can be fast-forwarded.
  (use "git pull" to update your local branch)
  
# and then

* 043e801 (origin/master) tacoworld: plenty o tacos
| * c5043f8 (origin/otr, otr) otr: what happens when over the rice is here?
|/  
* 7241432 (HEAD -> master) tacoworld: v cool good for you dan but I wanted tacos first bruh so back off
* 9c55fa3 tacoworld: I, dan, also desire tacos
* b115876 tacoworld: my name is Zach and I do not fear my hunger
* 29ae020 tacoworld: initial commit

```

our current state (`HEAD -> master`) has two paths it could go forward (sync up with `origin/master`, the remote branch, or move toward the `otr` branch.

the changes that dan made to the remote were implemented in our local remote branch `origin/master`. let's also incorporate those changes in our local `master` branch as well by using `git merge`

```sh
git merge origin/master
```

```
Updating 7241432..043e801
Fast-forward
 tacoworld.py | 2 +-
 1 file changed, 1 insertion(+), 1 deletion(-)
```

```sh
git log --oneline --decorate --graph --all
```

```
* 043e801 (HEAD -> master, origin/master) tacoworld: plenty o tacos
| * c5043f8 (origin/otr, otr) otr: what happens when over the rice is here?
|/  
* 7241432 tacoworld: v cool good for you dan but I wanted tacos first bruh so back off
* 9c55fa3 tacoworld: I, dan, also desire tacos
* b115876 tacoworld: my name is Zach and I do not fear my hunger
* 29ae020 tacoworld: initial commit
```

### ...why did I just do that?

okay, so, in practice you may just end up typing `git pull` all the time and letting `git` try and do the `fetch` (will always work!) and the `merge` (will only work if a fast forward merge is possible). usually, that's good enough.

**when `git pull` fails**, though, it will fail in the `merge` step. at that time, it's good to know what changed for you (your remote tracking branches) and what `git` is trying to do in that moment (`merge`).

also, sometimes you will have made local changes and you won't be sure if remote changes have been made yet. you can *always* run `git fetch` with *no* side effects, so if all you want to do is find out what the outside world is up to, run `git fetch` instead of `git pull`.

### one last `merge`

today's the day. I've had tacos for 100 days in a row (my model is extremely consistent, what can I say) and I see OTR outside.

I decide to deploy my OTR `branch` into production. I want to `merge` it into `master`.

take a second to think about this: I want to take the changes I made in the `otr` branch and incorporate them into the `master` branch.

**<div align="center">what should I do?</div>**

```sh
git checkout master
git merge otr
```

```
Auto-merging tacoworld.py
CONFLICT (content): Merge conflict in tacoworld.py
Automatic merge failed; fix conflicts and then commit the result.
```

<br><div align="center"><img src="https://media1.tenor.com/images/0a37ef8f52e2232d85a2070d56801987/tenor.gif?itemid=5026106" width=500></div>

### resolving conflicts

finally, at some point you will just have conflicts. at least with `git` you will know where they are.

`git` tried its very best to apply the changes I made in the `otr` branch on top of the current state of `master`, but it couldn't be done -- one of the lines in `master` (dan' line) has changed since I started `otr`. I have to resolve the conflict.

`git` lets us *know* that there is a conflict by editing the `tacoworld.py` file and adding markers to show us the "before and after" of our different code versions. the general format is

```
unchanged code...
unchanged code...
<<<<<<< HEAD
the version of the code
that is in the current
working state (i.e. HEAD)
=======
the version of the code
that is in the new branch
change state (i.e. otr)
>>>>>>> [the name of the branch or sha we are merging into HEAD]
more unchanged code...
more unchanged code...
...
```

in this particular instance, the conflict is the entire file:

```
<<<<<<< HEAD
print("I, Zach Lamberty, want tacos more than *anyone*, especially more than dan!")
print("I, dan fein, want tacos, so it's good there are enough to share!")
=======
print("Today, actually, I (Zach Lamberty), want OTR!")
print("I, dan fein, want tacos!")
>>>>>>> otr
```

+ the first block is the current state of `master`
    + we want to take the 2nd line (dan' edited line) from this block
+ the second block is the current state of `otr`
    + we want to take the 1st line (my edited line) from this block
    
crack open a text editor and actually make those edits. when you're confident you've done it correctly, remove the `<<<<<<<`, `=======`, and `>>>>>>>` lines.

the end result is:

```python
print("Today, actually, I (Zach Lamberty), want OTR!")
print("I, dan fein, want tacos, so it's good there are enough to share!")
```

normally after adding changes we would run `git add -p`. however, while resolving conflicts there is not `patch`ing to do -- just add the entire file

```sh
git add tacoworld.py
```

then, create a commit indicating that this was a `merge` commit which incorporated the `otr` branch:

```sh
git commit -m 'merging otr branch changes'
```

```
[master 7dc87e9] merging otr branch changes
```

look at what we did!

```sh
git log --oneline --decorate --graph --all
```

```
*   7dc87e9 (HEAD -> master) merging otr branch changes
|\  
| * c5043f8 (origin/otr, otr) otr: what happens when over the rice is here?
* | 043e801 (origin/master) tacoworld: plenty o tacos
|/  
* 7241432 tacoworld: v cool good for you dan but I wanted tacos first bruh so back off
* 9c55fa3 tacoworld: I, dan, also desire tacos
* b115876 tacoworld: my name is Zach and I do not fear my hunger
* 29ae020 tacoworld: initial commit
```

go ahead and `push` that out to the world

```sh
git push origin master
```

```
Counting objects: 3, done.
Delta compression using up to 4 threads.
Compressing objects: 100% (2/2), done.
Writing objects: 100% (3/3), 397 bytes | 397.00 KiB/s, done.
Total 3 (delta 0), reused 0 (delta 0)
To github.com:RZachLamberty/tacoworld.git
   043e801..7dc87e9  master -> master
```

## a second approach to collaboration: pull requests and forking

I mentioned wayyyyyyy back at the start of this lecture `github` allows you to add collaborators to your repo as one way of working together. I also suggested there was another -- that other way is very common and very cool.

`github` (not `git`, mind you!) supports a second type of workflow:

+ I own a main repo (in this case, `tacoworld`) and share it on `github`
+ you create a branch based off of my `github` master branch
    + this can happen in two ways!
        + you are a collaborator and do it the same way we showed above
        + you `fork` that repo -- create an independent copy of the repository that you control completely

<br><div align="center"><img src="http://drive.google.com/uc?export=view&id=1Lcv_LF2VFHk5eZlBGtZzp_iWNtIkX7Q-" width="850"></div>

+ you get your branch on your local machine (`clone` my repo that has your branch, or `clone` your `fork`ed copy)
+ you make your changes
+ you `push` them back up to `github`
+ `github` (not `git`!) can compare your `branch` to `master`
    + for single-repo situations it's obvious; for `fork` copies `github` is comparing changes you make to my original repo
+ `github` presents a button asking if you would like to create a `pull request`
+ this intiates an entirely-web-based and entirely-`github`-based process of asking me to approve the changes you made
    + companies put a lot of process here!

I won't dive into this now because we will cover it in depth on a future homework

## last resorts

[warning: language is NSFW, but it's assumed this is a resource when language is already being maximally deployed](https://ohshitgit.com/)

## follow-up resources

+ http://try.github.io/ *especially the "learn by doing"* sections!
+ rtfm: https://git-scm.com/doc
+ a short primer on `git merge`: https://dev.to/neshaz/how-to-use-git-merge-the-correctway-25pd

<div align="center"><code>git</code> along, little doggies</div>
<div align="center"><img align="middle" src="https://i.ytimg.com/vi/9cSpDCYo1GM/maxresdefault.jpg" width=600></div>

# END OF LECTURE


next lecture: [`python`](005_python.ipynb)